In [1]:
from os.path import join
import pandas as pd
import seaborn as sns
import warnings
import numpy as np
import pickle

from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.tree import DecisionTreeRegressor


from deep_aqi import ROOT


pd.set_option('expand_frame_repr', False)
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 25)
pd.options.mode.chained_assignment = None 

In [2]:
def MAPE(data):
    data = data[data['True'] > 0]
    return np.mean(np.abs(data['True'] - data.Pred) / data['True'])

In [3]:
def MdAPE(data):
    data = data[data['True'] > 0]
    return np.median(np.abs(data['True'] - data.Pred) / data['True'])

In [4]:
def save_experiment_result(model_name, param_name, save_var, save_var_name):
    file_name = f'{model_name}__{param_name}__{save_var_name}__'
    save_path = join(ROOT, 'jupyter', 'results', file_name)

    with open(save_path, 'wb') as f:
        pickle.dump(save_var, f, pickle.HIGHEST_PROTOCOL)

In [5]:
warnings.filterwarnings("ignore")

In [6]:
PROCESSED_DATA = join(ROOT, 'data', 'processed')

In [7]:
# file_path = join(PROCESSED_DATA, '88101.parquet')

# data_source = pd.read_parquet(file_path)
# target_column = data_source.columns[-1]

In [8]:
# data_source.head()

In [9]:
paths = ['88101.parquet', '88502.parquet', '81102.parquet', '44201.parquet', '42401.parquet', '42101.parquet', '42602.parquet']
model_name = 'linear_model_baseline'


for data_source_file in paths:
    file_path = join(PROCESSED_DATA, data_source_file)
  
    data_source = pd.read_parquet(file_path)
    target_column = data_source.columns[-1]
    
    to_concat = []
    for site_code, df in data_source.groupby(by='SiteCode'):

        df['Hour'] = df.LocalDate.dt.hour
        df = pd.concat([df, pd.get_dummies(df.Hour, prefix='Hour')], axis=1)
        df.drop('Hour', axis='columns', inplace=True)

        df['Weekday'] = df.LocalDate.dt.weekday
        df = pd.concat([df, pd.get_dummies(df.Weekday, prefix='Weekday')], axis=1)
        df.drop('Weekday', axis=1, inplace=True)

        df['Month'] = df.LocalDate.dt.month
        df.drop('Month', axis=1, inplace=True)


        Y = df.loc[:, target_column]
        df.drop(['SiteCode', target_column], axis=1, inplace=True)
        df.set_index('LocalDate', inplace=True)
        X = df.copy()

        Y = MinMaxScaler().fit_transform(Y.values.reshape(-1, 1))
        Y = pd.Series(Y.flatten(), index=X.index)  

        X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=23)


        pipe = Pipeline([('scaler', StandardScaler()), ('regressor', Ridge())])
        param_grid = {'scaler': [StandardScaler(), MinMaxScaler(), None],
                      'regressor': [Ridge(), DecisionTreeRegressor()],
                      }
        search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=3, verbose=2)
        search.fit(X_train, y_train)
        search.score(X_test, y_test)



        y_pred = search.predict(X_test)


        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)

        prediction = pd.DataFrame({'True': y_test, 'Pred': y_pred}, index=X_test.index)


        mape = MAPE(prediction)
        mdape = MdAPE(prediction)


        result = pd.DataFrame(index = [site_code],
                              data={'MAE': mae,
                                    'MSE': mse,
                                    'MAPE': mape,
                                    'MdAPE': mdape,
                                   })



        to_concat.append(result)
    results = pd.concat(to_concat)
    
    data_source_name = data_source_file.split('.')[0]
    save_experiment_result(model_name, data_source_name, results, 'Results')


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    2.2s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    1.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    2.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    2.7s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:    2.7s finished


In [10]:
results

,MAE,MSE,MAPE,MdAPE
Alabama_Jefferson_23.0,0.089306,0.019787,0.696562,0.392857
California_Fresno_5001.0,0.081645,0.012492,0.902238,0.445969
California_Kern_6001.0,0.065687,0.007435,0.574707,0.353574
California_Sacramento_11.0,0.045123,0.004139,0.586897,0.396496
California_Santa Barbara_2011.0,0.052668,0.005509,0.758244,0.399898
Colorado_Rio Blanco_5.0,0.036791,0.003133,0.472785,0.281619
Colorado_Rio Blanco_6.0,0.050238,0.005936,1.598669,0.697734
District Of Columbia_District of Columbia_43.0,0.069885,0.008745,0.680851,0.412009
Indiana_Lake_22.0,0.084210,0.012251,0.749477,0.363566
Indiana_Marion_78.0,0.079499,0.011058,0.885110,0.446758
